In [1]:
import Pkg
Pkg.activate("..")

# these lines are required only for local installations
Pkg.develop(path="../../scp_traj_opt/")
Pkg.precompile()

using SCPToolbox
using ECOS
using OSQP
using SCS
using Ipopt

  Activating project at `~/ACL/AAS/SCPToolbox_tutorial`
   Resolving package versions...
  No Changes to `~/ACL/AAS/SCPToolbox_tutorial/Project.toml`
  No Changes to `~/ACL/AAS/SCPToolbox_tutorial/Manifest.toml`


In [2]:
opts = Dict("verbose" => 0)
pbm = ConicProgram(solver = ECOS.Optimizer, solver_options = opts)

Conic linear program

  Feasibility problem
  0 variables (0 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    0 elements
    0 blocks


  Parameter argument
    0 elements
    0 blocks


In [3]:
v = @new_variable(pbm, 2, "z")

Vector variable
  2 elements
  (2,) shape
  Name: z
  Block index in stack: 1
  Indices in stack: 1:2
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     z[1]
     z[2]

In [4]:
eta = @new_variable(pbm,1,"eta")

Vector variable
  1 elements
  (1,) shape
  Name: eta
  Block index in stack: 2
  Indices in stack: 3
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     eta

In [ ]:
# ..:: Initialize ::..
ref = [0;0];

history = SCPHistory()

callback_fun! = pbm.traj.callback!
user_callback = !isnothing(callback_fun!)

# ..:: Iterate ::..
iter_max = 30
k = 1 # Iteration counter
while true
    # Construct the subproblem
    # spbm = Subproblem(pbm, k, ref)

    # Linearized nonconvex constraint
    # TODO
    
    # Hyperplane constraint
    # TODO
    
    # Halfspace constraint
    # TODO
    
    # Trust region
    # TODO
    
    # Buffer constraints
    # TODO
    
    # Cost function
    # TODO

    # Save history of the subproblem
    # TODO

    # Solve the subproblem
    # TODO


    # Check stopping criterion
    # z - ref <= eps_conv

    # Update reference trajectory
    # set reference to solution
   
    # Print iteration info
    # TODO - print_info(spbm)

    # Stop at maximum iterations
    k += 1
    if k > iter_max
        break
    end
end


In [23]:
# Convexified nonconvex constraint


a_ncvx = [-2*ref[1];1.0];
b_ncvx = -ref[1]^2;
params = Dict("n" => 2, "a" => a_ncvx, "b" => b_ncvx)
A = Matrix(transpose(params["a"]))
b = -params["b"]

@show A

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, ZERO, "my-hypln", (x,),
    begin
        local x, = arg
        A*x .+ b
    end)

A = [1.1 3.2]


Name: my-hypln
Cone f(x,p)∈K, where:
K is a zero cone, {z : z=0}
f(x,p) = 
  1.1 x[1] + 3.2 x[2] - 9
Affine function
Arguments:
  x (block 1) : 1:2


In [23]:
# Convex hyperplane constraint

params = Dict("n" => 2, "a" => [0.1;1], "b" => 0.6)
A = Matrix(transpose(params["a"]))
b = -params["b"]

@show A

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, ZERO, "my-hypln", (x,),
    begin
        local x, = arg
        A*x .+ b
    end)

A = [1.1 3.2]


Name: my-hypln
Cone f(x,p)∈K, where:
K is a zero cone, {z : z=0}
f(x,p) = 
  1.1 x[1] + 3.2 x[2] - 9
Affine function
Arguments:
  x (block 1) : 1:2


In [24]:
# Halfspace constraint

params = Dict("n" => 2, "a" => [1.0;0.0], "b" => 0.85)
A = Matrix(transpose(params["a"]))
b = -params["b"]

@show A*[1;2]

pbm = ConicProgram(params)

x = @new_variable(pbm, params["n"],"x")

# Define the constraint
cstr = @add_constraint(
    pbm, NONPOS, "my-hlfspace", (x,),
    begin
        local x, = arg
        println(size(A*x))
        A*x .+ b
    end)

A * [1; 2] = [7.5]
(1,)


Name: my-hlfspace
Cone f(x,p)∈K, where:
K is a nonpositive orthant cone, {z : z≤0}
f(x,p) = 
  1.1 x[1] + 3.2 x[2] - 9
Affine function
Arguments:
  x (block 1) : 1:2
